In [ ]:
library(tidyverse)
library(DESeq2)

In [ ]:
# TODO: Consider filtering counts (e.g. To remove rows with fewer than 10 reads)
cts <- read_delim(snakemake@input[["counts"]],delim = "\t", skip = 1) %>% 
    rename_with(., ~ stringr::str_extract(pattern = "(?<=\\/)\\w{4}-\\d{1}-\\w+-Seq(?=\\/)", string = .x), contains("out.bam")) %>% 
    column_to_rownames("Geneid") %>% 
    mutate(total_reads = rowSums(select(., contains("Seq")))) %>% 
    # filter(total_reads >= 10) %>% 
    dplyr::select(-Chr, -Start, -End, -Strand, -Length, -total_reads) %>% 
    as.matrix(.)

In [ ]:
head(cts)

In [ ]:
ugt_gene <- "ACLI19_g12275"
cyp73_gene <- "ACLI19_g12279"
cyp79_gene <- "ACLI19_g12294"
li_gene <- "ACLI19_g37877"

In [ ]:
cts[cyp73_gene,]

In [ ]:
samples <- read_delim(snakemake@config[["sample_sheet"]], delim = "\t")
samples

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = samples,
                              design = ~ cyanotype + tissue)
dds$group <- factor(paste0(dds$cyanotype, dds$tissue))
design(dds) <- ~ group
dds

In [ ]:
dds <- DESeq(dds)

In [ ]:
d <- plotCounts(dds, gene=li_gene, intgroup=c("tissue", "cyanotype"), returnData=TRUE)

li_plot <- ggplot(d, aes(x = tissue, y = count)) + 
    geom_point(aes(shape = cyanotype, color = cyanotype, fill = cyanotype), 
               size = 6, position=position_dodge(width = 0.5), alpha = 0.7) +
    ylab("Normalized read count") + xlab("Tissue") +
    scale_color_manual(values=c("#C81C00", "#0068FF")) +
    theme_classic() +
    ggtitle("Gene = Li") +
    theme(axis.text = element_text(size = 14),
          axis.title = element_text(size = 17),
          legend.text = element_text(size = 12),
          legend.title = element_text(size = 14),
          title = element_text(size = 18))
li_plot

ggsave(filename = snakemake@output[["li_plot"]], plot = li_plot, device = "pdf", width = 8, height = 8, dpi = 600, units = 'in')

In [ ]:
d <- plotCounts(dds, gene=ugt_gene, intgroup=c("tissue", "cyanotype"), returnData=TRUE)

ugt_plot <- ggplot(d, aes(x = tissue, y = count)) + 
    geom_point(aes(shape = cyanotype, color = cyanotype, fill = cyanotype), 
               size = 6, position=position_dodge(width = 0.5), alpha = 0.7) +
    ylab("Normalized read count") + xlab("Tissue") +
    scale_color_manual(values=c("#C81C00", "#0068FF")) +
    theme_classic() +
    ggtitle("Gene = UGT85K17") +
    theme(axis.text = element_text(size = 14),
          axis.title = element_text(size = 17),
          legend.text = element_text(size = 12),
          legend.title = element_text(size = 14),
          title = element_text(size = 18))
ugt_plot

ggsave(filename = snakemake@output[["ugt_plot"]], plot = ugt_plot, device = "pdf", width = 8, height = 8, dpi = 600, units = 'in')

In [ ]:
d <- plotCounts(dds, gene=cyp79_gene, intgroup=c("tissue", "cyanotype"), returnData=TRUE)

cyp79_plot <- ggplot(d, aes(x = tissue, y = count)) + 
    geom_point(aes(shape = cyanotype, color = cyanotype, fill = cyanotype), 
               size = 6, position=position_dodge(width = 0.5), alpha = 0.7) +
    ylab("Normalized read count") + xlab("Tissue") +
    scale_color_manual(values=c("#C81C00", "#0068FF")) +
    theme_classic() +
    ggtitle("Gene = CYP79D15") +
    theme(axis.text = element_text(size = 14),
          axis.title = element_text(size = 17),
          legend.text = element_text(size = 12),
          legend.title = element_text(size = 14),
          title = element_text(size = 18))
cyp79_plot

ggsave(filename = snakemake@output[["cyp79_plot"]], plot = cyp79_plot, device = "pdf", width = 8, height = 8, dpi = 600, units = 'in')

In [ ]:
d <- plotCounts(dds, gene=cyp73_gene, intgroup=c("tissue", "cyanotype"), returnData=TRUE)

cyp73_plot <- ggplot(d, aes(x = tissue, y = count)) + 
    geom_point(aes(shape = cyanotype, color = cyanotype, fill = cyanotype), 
               size = 6, position=position_dodge(width = 0.5), alpha = 0.7) +
    ylab("Normalized read count") + xlab("Tissue") +
    scale_color_manual(values=c("#C81C00", "#0068FF")) +
    theme_classic() +
    ggtitle("Gene = CYP736A187") +
    theme(axis.text = element_text(size = 14),
          axis.title = element_text(size = 17),
          legend.text = element_text(size = 12),
          legend.title = element_text(size = 14),
          title = element_text(size = 18))
cyp73_plot

ggsave(filename = snakemake@output[["cyp73_plot"]], plot = cyp73_plot, device = "pdf", width = 8, height = 8, dpi = 600, units = 'in')